www.titech.ac.jp/0/education/graduate-majors/is  
1. URLを文字単位に分解する  
| w | w | w | . | t | i | t | e | c | h | ...
2. 各文字をASCIIコードの16進表記に変換する  
['0x77', '0x77', '0x77', '0x2e', '0x74', '0x69', '0x74', '0x65' ...
3. ホスト部、パス部それぞれの先頭から、4ビットずつシフトして、出現する値を列挙する 各文字に4bitずつ間を開ける
例：12345678→135,246,357,468
['777', '777',
4. ホスト部、パス部それぞれに出現した値（0x00から0xFF）の個数を数え、4096次元のベクトルにする
5. この二つを連結して、8192次元のベクトルにする
6. ベクトルを正規化する

メモ：
- 正規化するとURL長の情報は失われるが、含まれる値の種類数の大小で判断できる  
- 4ビットずつ動かしているので、URLの並びの情報も保持できる  
二文字のつながりの部分を残して置ける

ホスト部とパス部分ける理由
- この二つは出てくる文字列の性質が違う  
ホスト、ドメインとディレクトリ、ファイル

In [10]:
import numpy as np
import math

def url_to_vector(url):
    # プロトコル名を削除
    if "://" in url:
        url = url.split("://", 1)[1]

    # URLをホスト部とパス部に分割
    parts = url.split('/', 1)
    host = parts[0]
    path = parts[1] if len(parts) > 1 else ""  # '/'がない場合は空文字列を設定


    # 文字単位に分割してASCIIコードの16進表記に変換
    host_hex = [hex(ord(c)) for c in host]
    path_hex = [hex(ord(c)) for c in path]
   # print(host_hex)

    # 0xを取り除いて連結
    host_str = ''.join([h[2:] for h in host_hex])  # '0x'を取り除いて連結
    path_str = ''.join([p[2:] for p in path_hex])  # '0x'を取り除いて連結
   # print(host_str)

    # 4ビットずつ右にシフトを繰り返し、出現する値をリストで列挙
    host_values = [host_str[i] + host_str[i+2] + host_str[i+4] for i in range(0, len(host_str) - 4)]
    path_values = [path_str[i] + path_str[i+2] + path_str[i+4] for i in range(0, len(path_str) - 4)]


   # print(host_values)

    # 出現した値の個数を数える
    def count_values(values):
        count_vector = [0] * 4096
        for v in values:
            index = int(v, 16)
            count_vector[index] += 1
        return count_vector

    host_vector = count_values(host_values)
    path_vector = count_values(path_values)

    # 2つのベクトルを連結
    combined_vector = host_vector + path_vector

    # ベクトルを正規化
    norm = math.sqrt(sum([x**2 for x in combined_vector]))
    normalized_vector = [x/norm for x in combined_vector]

    return normalized_vector

# 使用例
url = "www.titech.ac.jp/0/education/graduate-majors/is"
normalized_vector = url_to_vector(url)
print("Vector:", normalized_vector)

Vector: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

モデルの定義

In [11]:
pip install chainer

In [12]:
from chainer import Chain
import chainer.functions as F
import chainer.links as L
class Model(Chain):
      def __init__(self):
          super(Model, self).__init__()
          with self.init_scope():
               self.l1 = L.Linear(None, 256)
               self.l2 = L.Linear(None, 256)
               self.l3 = L.Linear(None, 2)
      def __call__(self, x):
          h1 = F.dropout(F.relu(self.l1(x)),
                        ratio=0.75)
          h2 = F.dropout(F.relu(self.l2(h1)),
                        ratio=0.75)
          y = self.l3(h2)
          return y

# 実験

データセットとして以下のものを用いる

https://www.kaggle.com/datasets/sid321axn/malicious-urls-dataset?resource=download


- 良性URL: 428103個
- 改ざんURL:  96457個
- フィッシングURL: 94111個
- マルウェアURL: 32520個

の合計651,191個のデータセット

複数のデータセットを利用

- ISCX-URL-2016
- マルウェアドメインブラックリストのデータセット
- Phishtankデータセット
- PhishStormデータセット

今回は良性か悪性かのみ識別する二値分類問題として考える(良性URLは0, それ以外は1)

In [13]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

%cd /content/drive/My Drive/Colab Notebooks

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [14]:
from chainer import Chain, datasets, iterators, optimizers, training
from chainer.training import extensions
import chainer.functions as F
import chainer.links as L
import numpy as np
import pandas as pd

# CSVファイルからデータを読み込む
data_path = "malicious_phish.csv"
df = pd.read_csv(data_path)
print(df)

                                                      url        type
0                                        br-icloud.com.br    phishing
1                     mp3raid.com/music/krizz_kaliko.html      benign
2                         bopsecrets.org/rexroth/cr/1.htm      benign
3       http://www.garage-pirenne.be/index.php?option=...  defacement
4       http://adventure-nicaragua.net/index.php?optio...  defacement
...                                                   ...         ...
651186            xbox360.ign.com/objects/850/850402.html    phishing
651187       games.teamxbox.com/xbox-360/1860/Dead-Space/    phishing
651188         www.gamespot.com/xbox360/action/deadspace/    phishing
651189      en.wikipedia.org/wiki/Dead_Space_(video_game)    phishing
651190          www.angelfire.com/goth/devilmaycrytonite/    phishing

[651191 rows x 2 columns]


In [15]:
# DataFrameからランダムに60000行を取得
df = df.sample(n=6000, random_state=42)

print(df)

                                                      url        type
536448             http://37.49.226.178/deusbins/deus.sh4     malware
40630   medical-dictionary.thefreedictionary.com/Galt+...      benign
630496                         www.jscape.com/sshfactory/    phishing
426724  http://www.wsnc.org.au/component/jcalpro/view/983  defacement
184034  virtualtourist.com/travel/North_America/Canada...      benign
...                                                   ...         ...
306810        airforce.forces.gc.ca/v2/page-eng.asp?id=30      benign
463468     http://batutik.com.ua/produkcziya/bassein.html  defacement
348007  http://manola-souvanlasy.com/index.php?view=ar...  defacement
425035                          myspace.com/locusmagazine      benign
353876                starwars.wikia.com/wiki/Rick_Parker      benign

[6000 rows x 2 columns]


In [16]:
# 'label'列の追加 benign(良性)のものは0, それ以外(悪性)のものは1を付与
df['label'] = df['type'].apply(lambda x: 0 if x == 'benign' else 1)

urls = df['url'].tolist()
labels = df['label'].tolist()  # 0: 良性, 1: 悪性

In [17]:
# URLをベクトル化・正規化
vectors = np.array([url_to_vector(url) for url in urls], dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# データセットの作成
dataset = datasets.TupleDataset(vectors, labels)

# データセットをトレーニング用と検証用に分割
train_dataset, valid_dataset = datasets.split_dataset_random(dataset, int(len(dataset)*0.8), seed=0)

# イテレータの設定
train_iter = iterators.SerialIterator(train_dataset, batch_size=32)
valid_iter = iterators.SerialIterator(valid_dataset, batch_size=32, repeat=False, shuffle=False)

# ニューラルネットモデルのインスタンス化
model = L.Classifier(Model())

# オプティマイザの設定
optimizer = optimizers.Adam()
optimizer.setup(model)

# トレーニングの設定
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'))

# 検証用のイテレータとモデルを使用して、検証の精度と損失を計算するextensionを追加
trainer.extend(extensions.Evaluator(valid_iter, model))

# ログを出力するextensionを追加
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))

# トレーニングの実行
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           0.521565    0.346368              0.734167       0.85773                   
2           0.319065    0.305168              0.875208       0.875                     
3           0.246509    0.308156              0.903125       0.877467                  
4           0.199372    0.308133              0.922917       0.880757                  
5           0.165661    0.319042              0.93625        0.876645                  
6           0.129722    0.349412              0.952083       0.875                     
7           0.109503    0.429203              0.963542       0.869243                  
8           0.091049    0.404961              0.965833       0.881579                  
9           0.0660389   0.455407              0.976042       0.875822                  
10          0.0596333   0.516035              0.978333       0.868421                  
11          0.0475556   0.521388  

In [18]:
char = 'a'
print(ord(char))
print(hex(ord(char)))
chr(10)

97
0x61


'\n'